In [1]:
YEAR, EVALUATION = 2020, 3

In [2]:
from IPython.core.display import display, HTML
display(HTML(f'<h1>ANÁLISIS DE LA {EVALUATION}ª EVALUACIÓN</h1>'))
display(HTML(f'<h2>IES Puerto de la Cruz - Telesforo Bravo | Curso {YEAR}-{YEAR + 1}</h2>'))

- [CARGA DE DATOS](#CARGA-DE-DATOS)
- [NÚMERO DE ALUMNADO, PROFESORADO Y RATIOS](#NÚMERO-DE-ALUMNADO,-PROFESORADO-Y-RATIOS)
- [ÉXITO ESCOLAR](#%C3%89XITO-ESCOLAR)
- [ABSENTISMO](#ABSENTISMO)
- [GESTIÓN DE LA CONVIVENCIA](#GESTIÓN-DE-LA-CONVIVENCIA)
- [GRUPOS MÁGICOS](#GRUPOS-MÁGICOS)

In [3]:
import os
import sys
import re
from pathlib import Path

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from matplotlib import pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import colorlover as cl
from IPython.core.display import display, HTML

PROJECT_DIR = Path('../..').resolve()

sys.path.append(str(PROJECT_DIR))
from services import loaders, charts, myplotly, utils

init_notebook_mode(connected=True)

Estilo propio para los dataframes:

In [4]:
with open(PROJECT_DIR / 'custom.css') as f:
    css_rules = ''.join(f.readlines())
HTML('<style>' + css_rules + '</style>')

Deshabilitar el *auto-scrolling* en todo el notebook:

In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## CARGA DE DATOS

In [6]:
df, df_bc, labels = loaders.load_data(YEAR, EVALUATION)

In [7]:
df.head()

etapa  éxito  absentismo_justificado  \
curso evaluación grupo                                        
C1819 E1         ESO1A   ESO   20.0                    0.66   
                 ESO1B   ESO   47.4                    0.14   
                 ESO1C   ESO   28.0                    1.68   
                 ESO2A   ESO   42.9                    2.55   
                 ESO2B   ESO   42.9                    0.83   

                        absentismo_injustificado  partes  \
curso evaluación grupo                                     
C1819 E1         ESO1A                      3.16     4.0   
                 ESO1B                      6.02     5.0   
                 ESO1C                      4.71     1.0   
                 ESO2A                      3.84     2.0   
                 ESO2B                      5.46     3.0   

                        suspensión_asistencia  ratio  absentismo  éxito_abs  \
curso evaluación grupo                                                        
C1819 E1         ESO1A                    1.0     20        3.82        4.0   
                 ESO1B                    0.0     19        6.16        9.0   
                 ESO1C                    0.0     25        6.39        7.0   
                 ESO2A                    0.0     28        6.39       12.0   
                 ESO2B                    1.0     28        6.29       12.0   

                            ccbb  
curso evaluación grupo            
C1819 E1         ESO1A  3.964286  
                 ESO1B  4.803571  
                 ESO1C  4.257143  
                 ESO2A  4.807143  
                 ESO2B  4.825000

In [8]:
df_bc.head()

nivel  item  PA  AD  MA  EX  marca
curso evaluación grupo                                   
C1819 E1         ESO1A  1ESO    CL  60  40   0   0  3.500
                 ESO1A  1ESO  CMCT  60  30  10   0  3.750
                 ESO1A  1ESO    CD  30  70   0   0  4.250
                 ESO1A  1ESO   AAP  55  35  10   0  3.875
                 ESO1A  1ESO   CSC  45  55   0   0  3.875

In [9]:
# recurrent variables
stages = df['etapa'].unique()
years = df.index.levels[0]
current_year, current_eval = labels[-1]

In [10]:
# dataframe for current year and evaluation
dfc = df.loc[labels[-1]]
dfc_bc = df_bc.loc[labels[-1]]

In [11]:
# dataframe for comparisons with last evaluation
df_comp = dfc.merge(df.loc[labels[-2]], left_index=True, right_index=True)

df_bc_last_eval_new_index = df_bc.loc[labels[-2]].reset_index().set_index(['grupo', 'item'])
dfc_bc_new_index = dfc_bc.reset_index().set_index(['grupo', 'item'])
df_bc_comp = dfc_bc_new_index.merge(df_bc_last_eval_new_index, left_index=True, right_index=True)
df_bc_comp = df_bc_comp.reset_index().set_index(['grupo'])

# NÚMERO DE ALUMNADO, PROFESORADO Y RATIOS

In [12]:
# filter for current evaluation
df_ce = df.xs(current_eval, level='evaluación')

## Evolución del número total de alumnado y profesorado

In [13]:
extra_historic_year = loaders.get_year_label(YEAR - 3)

In [14]:
years_aux    = [extra_historic_year] + list(years)       # Vamos a añadir un año extra a la comparativa
students     = [    731] + list(df.query(f'evaluación == "{labels[-1][1]}"').groupby('curso').agg({'ratio': 'sum'})['ratio'].values)             
students_dam = (      0,   0,   0,  27)      # Estudiantes a resaltar en cada curso (se restan al global)
teachers     = (     71,  70,  78,  86)      # Valores de profesorado extraídos de Tifón: 2011/12:70, 2012/13:68, 2013/14:68, 2015/16:70, 2016/17:69, 2017/18:71, 2018/19:70, 2019/20:78, 2020/21:86
myplotly.total_students_teachers_evolution(students, teachers, years_aux, students_diff=(students_dam, 'Alumnado DAM'))

## Evolución del número total de alumnado por niveles

In [15]:
myplotly.num_students_evolution(df_ce, stages, years)

## Evolución de la ratio de alumnado por etapas (media)

In [16]:
myplotly.num_students_evolution(df_ce, stages, years, ratio=True)

## Evolución de la ratio de alumnado por etapas (máximo)

In [17]:
myplotly.num_students_evolution(df_ce, stages, years, ratio=True, max_ratio=True)

## Evolución del número de grupos totales

In [18]:
series = []
for year in years:
    series.append(df.query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['etapa'].count())

myplotly.bar_simple(years, series, is_percentage=False)

## Evolución del número de grupos por etapas

In [19]:
series = {}
for year in years:
    series[year] = []
    for stage in stages:
        series[year].append(df.query(f'etapa == "{stage}" & curso == "{year}" & evaluación == "{labels[-1][1]}"')['etapa'].count())

myplotly.cbar(stages, series, is_percentage=False)

## Evolución del número de grupos en ESO y Bachillerato

In [20]:
etapas_niveles = {'ESO': 4, 'BACH': 2}
series = {}
titles = []
no_titles = True
for year in years:
    series[year] = []
    for etapa in etapas_niveles:
        for num in range(etapas_niveles[etapa]):
            series[year].append(df.query(f'grupo.str.contains("{num+1}") & etapa == "{etapa}" & curso == "{year}" & evaluación == "{labels[-1][1]}"', engine="python").count()['etapa'])
            if no_titles:
                titles.append(str(num+1)+etapa)
    no_titles = False

myplotly.cbar(titles, series, is_percentage=False)


# ÉXITO ESCOLAR

<div class="alert alert-success" role="alert">
    ÉXITO ESCOLAR = <b>0 suspensos</b>
</div>

## ÉXITO EN VALORES ABSOLUTOS

In [21]:
myplotly.hbar((dfc['ratio'], dfc['éxito_abs']), dfc.index, trace_names=('Ratio', 'Éxito'))

## ESO

In [22]:
dfc_ESO = loaders.get_data_by_stages(dfc, 'ESO')
myplotly.bar_simple(dfc_ESO.index, dfc_ESO['éxito'])

In [23]:
dfc_ESO.mean()

éxito                       52.875000
absentismo_justificado       1.628125
absentismo_injustificado     3.785625
partes                       3.500000
suspensión_asistencia        0.250000
ratio                       20.125000
absentismo                   5.413750
éxito_abs                   10.812500
ccbb                         5.154464
dtype: float64

## FPB

In [24]:
dfc_FPB = loaders.get_data_by_stages(dfc, 'FPB')
myplotly.bar_simple(dfc_FPB.index, dfc_FPB['éxito'])

In [25]:
dfc_FPB.mean()

éxito                       60.850
absentismo_justificado       5.195
absentismo_injustificado    20.575
partes                       5.500
suspensión_asistencia        1.500
ratio                        8.500
absentismo                  25.770
éxito_abs                    4.500
ccbb                           NaN
dtype: float64

## BACHILLERATO

In [26]:
dfc_BACH = loaders.get_data_by_stages(dfc, 'BACH')
myplotly.bar_simple(dfc_BACH.index, dfc_BACH['éxito'])

In [27]:
dfc_BACH.mean()

éxito                       65.220
absentismo_justificado       1.818
absentismo_injustificado     4.900
partes                       1.400
suspensión_asistencia        0.400
ratio                       18.200
absentismo                   6.718
éxito_abs                   12.000
ccbb                           NaN
dtype: float64

## CICLOS FORMATIVOS DE GRADO MEDIO

In [28]:
dfc_CFGM = loaders.get_data_by_stages(dfc, 'CFGM')
myplotly.bar_simple(dfc_CFGM.index, dfc_CFGM['éxito'])

In [29]:
dfc_CFGM.mean()

éxito                       80.116667
absentismo_justificado       1.526667
absentismo_injustificado    14.843333
partes                       0.000000
suspensión_asistencia        0.000000
ratio                       12.166667
absentismo                  16.370000
éxito_abs                    9.166667
ccbb                              NaN
dtype: float64

## CICLOS FORMATIVOS DE GRADO SUPERIOR

In [30]:
dfc_CFGS = loaders.get_data_by_stages(dfc, 'CFGS')
myplotly.bar_simple(dfc_CFGS.index, dfc_CFGS['éxito'])

In [31]:
dfc_CFGS.mean()

éxito                       81.084615
absentismo_justificado       0.883846
absentismo_injustificado     9.265385
partes                       0.000000
suspensión_asistencia        0.000000
ratio                       18.307692
absentismo                  10.149231
éxito_abs                   14.384615
ccbb                              NaN
dtype: float64

## GLOBAL

In [32]:
dfc_summary = dfc.groupby('etapa').mean().loc[stages]
myplotly.bar_simple(dfc_summary.index, dfc_summary['éxito'])

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [33]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).mean().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['éxito'].values
          for year in years}

myplotly.cbar(stages, series)

In [34]:
mean_success = dfc['éxito'].mean()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Éxito global medio: <b>{mean_success:.2f}</b>%
</div>
'''))

In [35]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR</h2>'))

    aux = df_comp['éxito_x'] - df_comp['éxito_y']
    myplotly.dbar(aux.index, aux)

## COMPETENCIAS BÁSICAS

**AAP**: Aprender a aprender.  
**CD**: Competencia digital.  
**CEC**: Conciencia y expresiones culturales.  
**CL**: Comunicación lingüística.  
**CMCT**: Competencia matemática y competencias básicas en ciencia y tecnología.  
**CSC**: Competencias sociales y cívicas.  
**SIEE**: Sentido de iniciativa y espíritu emprendedor.  

#### Valor resumen por competencia básica:
\begin{equation}
MARCA = 0.025 * \mathbb{PA} + 0.050 * \mathbb{AD} + 0.075 * \mathbb{MA} + 0.1 * \mathbb{EX}
\end{equation}

, donde $\mathbb{PA}$, $\mathbb{AD}$, $\mathbb{MA}$ y $\mathbb{EX}$ son los porcentajes de adquisición de la competencia básica en cuestión.

### ESTUDIO POR NIVELES Y COMPETENCIA

In [36]:
myplotly.bc_bar(dfc_bc)

## Análisis simplificado por competencias básicas

In [37]:
dfc_bc_marked = dfc_bc.groupby(['nivel', 'item']).mean().reset_index().pivot('item', 'nivel', 'marca')

In [38]:
colors = cl.scales['4']['div']['RdYlGn']
q1 = np.quantile(dfc_bc_marked.values, q=0.1)
q2 = 5
q3 = np.quantile(dfc_bc_marked.values, q=0.9)
if q3 > q2:
    new_thresholds = [q1, q2, q3]
else:
    new_thresholds = [q1, q2]
thresholds = utils.normalize_thresholds(dfc_bc_marked, new_thresholds)
colorscale = utils.make_colorscale(colors, thresholds, discrete=True)

myplotly.heatmap(dfc_bc_marked.index, dfc_bc_marked.columns, dfc_bc_marked.values.T, colorscale)

### Nivel competencial resumen por nivel

In [39]:
aux = dfc_bc['marca'].groupby(dfc_bc['nivel']).mean()
myplotly.bar_simple(aux.index, aux, is_percentage=False, yaxis_range=[0, 10], mark_colors=True)

### Nivel competencial resumen por competencias básicas

In [40]:
aux = dfc_bc['marca'].groupby(dfc_bc['item']).mean()
myplotly.bar_simple(aux.index, aux, is_percentage=False, yaxis_range=[0, 10], mark_colors=True)

### Máximo nivel competencial

In [41]:
dfc_bc.iloc[dfc_bc.reset_index()['marca'].idxmax()][['item', 'marca']]

item       CSC
marca    6.825
Name: ESO4C, dtype: object

### Mínimo nivel competencial

In [42]:
dfc_bc.iloc[dfc_bc.reset_index()['marca'].idxmin()][['item', 'marca']]

item     CMCT
marca    3.65
Name: ESO1D, dtype: object

## Evolución de los niveles competenciales

In [43]:
# filter by evaluation
df_bc_same_eval = df_bc.xs(labels[-1][1], level='evaluación')
myplotly.bc_evolution(df_bc_same_eval)

In [44]:
if EVALUATION > 1:
    display(HTML('<h2>Evolución competencial con respecto a la evaluación anterior</h2>'))
    myplotly.bc_diff(df_bc_comp)

## Correlación entre competencias básicas

> Se considera que dos variables cuantitativas están correlacionadas cuando los valores de una de ellas varían sistemáticamente con respecto a los valores homónimos de la otra: si tenemos dos variables (A y B) existe correlación entre ellas si al disminuir los valores de A lo hacen también los de B y viceversa. La correlación entre dos variables no implica, por sí misma, ninguna relación de causalidad.

Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Correlaci%C3%B3n)

In [45]:
aux = dfc_bc.reset_index().pivot(index='grupo', columns='item', values='marca').corr()
aux.values[np.triu_indices(aux.shape[0])] = np.NaN

colorscale = utils.make_colorscale(cl.scales['8']['div']['PiYG'])
myplotly.heatmap(aux.index, aux.columns, aux.values.T, colorscale=colorscale)

## Mejores y peores valoraciones de competencias básicas por grupo

In [46]:
myplotly.bc_extremes(dfc_bc)

# ABSENTISMO

<div class="alert alert-success" role="alert">
    ABSENTISMO = <b>Faltas justificadas + Faltas injustificadas</b>
</div>

**Se va a estudiar el ABSENTISMO ACUMULADO desde el 1 de septiembre hasta la fecha de finalización del trimestre de análisis.**

## ABSENTISMO TOTAL vs ABSENTISMO JUSTIFICADO

In [47]:
myplotly.hbar((dfc['absentismo'], dfc['absentismo_justificado']), dfc.index,
              trace_names=('Abs. Total (%)', 'Abs. Justificado (%)'))

## ESO

In [48]:
myplotly.bar_simple(dfc_ESO.index, dfc_ESO['absentismo'], colormap='Reds')

# FPB

In [49]:
myplotly.bar_simple(dfc_FPB.index, dfc_FPB['absentismo'], colormap='Reds')

## BACHILLERATO

In [50]:
myplotly.bar_simple(dfc_BACH.index, dfc_BACH['absentismo'], colormap='Reds')

## CICLOS FORMATIVOS DE GRADO MEDIO

In [51]:
myplotly.bar_simple(dfc_CFGM.index, dfc_CFGM['absentismo'], colormap='Reds')

## CICLOS FORMATIVOS DE GRADO SUPERIOR

In [52]:
myplotly.bar_simple(dfc_CFGS.index, dfc_CFGS['absentismo'], colormap='Reds')

## GLOBAL

In [53]:
myplotly.bar_simple(dfc_summary.index, dfc_summary['absentismo'], colormap='Reds')

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [54]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).mean().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['absentismo'].values
          for year in years}

myplotly.cbar(stages, series)

In [55]:
mean_absence = dfc['absentismo'].mean()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Absentismo global medio: <b>{mean_absence:.2f}</b>%
</div>
'''))

In [56]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR'))

    aux = df_comp['absentismo_x'] - df_comp['absentismo_y']
    myplotly.dbar(aux.index, aux, inverted_colors=True)

# GESTIÓN DE LA CONVIVENCIA

## PARTES DE GESTIÓN

In [57]:
dfc_reports = dfc.query('partes > 0')['partes']
myplotly.bar_simple(dfc_reports.index, dfc_reports, colormap='Reds', is_percentage=False)

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [58]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).sum().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['partes'].values
          for year in years}

myplotly.cbar(stages, series, is_percentage=False)

In [59]:
total_reports = dfc_reports.sum()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Total de partes de gestión: <b>{total_reports:.0f}</b>
</div>
'''))

In [60]:
if EVALUATION > 1:
    display(HTML('<h2>EVOLUCIÓN DE GRUPOS CON RESPECTO A LA EVALUACIÓN ANTERIOR'))

    aux = df_comp['partes_x'] - df_comp['partes_y']
    myplotly.dbar(aux.index, aux, is_percentage=False, inverted_colors=True)

## SUSPENSIÓN DEL DERECHO DE ASISTENCIA

In [61]:
dfc_non_attendance = dfc.query('suspensión_asistencia > 0')['suspensión_asistencia']
myplotly.bar_simple(dfc_non_attendance.index, dfc_non_attendance, colormap='Reds', is_percentage=False)

## COMPARATIVA CON LOS ÚLTIMOS CURSOS

In [62]:
aux = df.groupby(['etapa', 'curso', 'evaluación']).sum().reset_index().set_index('etapa')
series = {year: aux.loc[stages].query(f'curso == "{year}" & evaluación == "{labels[-1][1]}"')['suspensión_asistencia'].values
          for year in years}

myplotly.cbar(stages, series, is_percentage=False)

In [63]:
total_non_attendance = dfc_non_attendance.sum()
display(HTML(f'''
<div class='alert alert-info' role="alert">
    Total de partes de gestión con <b>suspensión</b> del derecho de asistencia: <b>{total_non_attendance:.0f}</b>
</div>
'''))

# RELACIÓN `éxito-competencias básicas`

> **NOTA**: Análisis únicamente con grupos de la ESO.

In [64]:
myplotly.scatter(dfc_ESO['éxito'], dfc_ESO['ccbb'], dfc_ESO.index,
        x_title='Éxito', y_title='Competencias básicas')

In [65]:
corr = dfc_ESO.corrwith(dfc['éxito'])['ccbb']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo éxito y alta adquisición de competencias básicas

In [66]:
dfc_ESO[(dfc_ESO['éxito'] < 20) & (dfc_ESO['ccbb'] > 5)][['éxito', 'ccbb']]

,éxito,ccbb
grupo,,


### Alto éxito y baja adquisición de competencias básicas

In [67]:
dfc_ESO[(dfc_ESO['éxito'] > 50) & (dfc_ESO['ccbb'] < 4)][['éxito', 'ccbb']]

,éxito,ccbb
grupo,,


# RELACIÓN `ratio-éxito`

In [68]:
myplotly.scatter(dfc['ratio'], dfc['éxito'], dfc.index,
        x_title='Ratio', y_title='Éxito')

In [69]:
corr = dfc.corrwith(dfc['ratio'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Baja ratio y bajo éxito

In [70]:
dfc[(dfc['ratio'] < 20) & (dfc['éxito'] < 20)][['ratio', 'éxito']]

,ratio,éxito
grupo,,


### Alta ratio y alto éxito

In [71]:
dfc[(dfc['ratio'] > 25) & (dfc['éxito'] > 50)][['ratio', 'éxito']]

,ratio,éxito
grupo,,
1ASR,28,71.4
2GIT,26,100.0


# RELACIÓN `absentismo-éxito`

In [72]:
myplotly.scatter(dfc['absentismo'], dfc['éxito'], dfc.index,
        x_title='Absentismo', y_title='Éxito')

In [73]:
corr = dfc.corrwith(dfc['absentismo'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo absentismo y bajo éxito

In [74]:
dfc[(dfc['absentismo'] < 10) & (dfc['éxito'] < 15)][['absentismo', 'éxito']]

,absentismo,éxito
grupo,,


### Alto absentismo y alto éxito

In [75]:
dfc[(dfc['absentismo'] > 25) & (dfc['éxito'] > 50)][['absentismo', 'éxito']]

,absentismo,éxito
grupo,,
2FPB,28.37,80.0


# RELACIÓN `partes-éxito`

In [76]:
myplotly.scatter(dfc['partes'], dfc['éxito'], dfc.index,
        x_title='Partes de gestión', y_title='Éxito')

In [77]:
corr = dfc.corrwith(dfc['partes'])['éxito']
display(HTML(f'''
    <div class='alert alert-success' role='alert'>
        COEFICIENTE DE CORRELACIÓN: <b>{corr:.4f}</b>
    </div>
'''))

## Detección de *outliers*

### Bajo número de partes y bajo éxito

In [78]:
dfc[(dfc['partes'] < 10) & (dfc['éxito'] < 10)][['partes', 'éxito']]

,partes,éxito
grupo,,


### Alto número de partes y alto éxito

In [79]:
dfc[(dfc['partes'] > 15) & (dfc['éxito'] > 30)][['partes', 'éxito']]

,partes,éxito
grupo,,


# GRUPOS MÁGICOS

> Son aquellos que tienen mejores valores de éxito escolar, menor absentismo y menos partes de gestión.

\begin{equation}
magic = 0.6 · success + 0.2 · e^{-absence} + 0.2 · e^{-reports}
\end{equation}

In [80]:
df2 = dfc.copy()
df2['magic'] = 0.6 * df2['éxito'] + 0.2 * \
    np.exp(-df2['absentismo']) + 0.2 * np.exp(-df2['partes'])
magic = df2.sort_values('magic', ascending=False)

series = {
    'Éxito escolar': magic['éxito'].values,
    'Absentismo': magic['absentismo'].values,
    'Partes gestión': magic['partes'].values
}

myplotly.cbar(magic.index, series, is_percentage=False, barmode='stack')

In [81]:
if EVALUATION > 1:
    display(HTML('<h1>PRUEBAS DE CERTIFICACIÓN PARA POBLACIÓN ESCOLAR</h1>'))
    myplotly.bar_pcei(labels[-1][0])